
# Fun with Word Embeddings

In [0]:
import warnings


warnings.filterwarnings("ignore")

### Установка библиотек
(For colab, or linux)

In [0]:
!pip3 install --upgrade nltk gensim

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

### Скачиваем данные
(для тех у кого linux), у кого windows скачайте файлы вручную и поместите в папку с проектом.

In [0]:
!wget -O p5.txt "https://raw.githubusercontent.com/king-menin/nlp-hse-winter2018/master/lecture%202.%20word%20vectors/p5.txt"

Word2vec pretrained

In [0]:
!wget -O ru.tar.gz "https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz?dl=1"

In [0]:
!tar -xvzf ru.tar.gz

nltk russian sent tokenizer

In [0]:
!wget -O russian.pickle https://raw.githubusercontent.com/mhq/train_punkt/master/russian.pickle

### 0. Чтение данных

In [0]:
with open("p5.txt") as file:
  data = file.read()

### 1. Токенизация

In [0]:
from nltk.tokenize import WordPunctTokenizer
import nltk
sent_tokenizer = nltk.data.load('russian.pickle')

tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(sent_tokenizer.sentences_from_text(data[:1000])[0]))

#### Задание:

* Удалите пустые строки. Пока не убедитесь, что пустые строки не удалены, не запускайте ru_sent_tokenize.
* приведите все к нижнему регистру и извлеките токены с помощью tokenizer. 
* data_tok должна содержать список токенов для каждого предложения в файле.

In [0]:
len(data)

In [0]:
import re

data = <your code here>
data_tok = <your code here>

In [0]:
# Test
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('а' <= x.lower() <= 'я' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [0]:
print(data_tok[0])

### 2. Word vectors
Есть несколько способов обучения word embeddings. Word2Vec и GloVe с различными целевыми функциями. Или fasttext, который использует уровень символов.

Выбор огромен, поэтому давайте начнем с малого, gensim еще одна библиотека nlp, которая включает в себя множество векторных моделей, в том числе word2vec.

In [0]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=1,  # consider words that occured at least 5 times
                 window=5, iter=30).wv  # define context as a 5-word window around the target word

In [0]:
# Теперь можем получить вектор !
model.get_vector('человек')

In [0]:
# или запросить похожие слова. Let's play!
model.most_similar('человек')

### 3. Визуализация векторов слов
Один из способов проверить, хороши ли наши векторы, - построить их. Дело в том, что эти векторы находятся в 3D + пространстве, и мы, люди, более привыкли к 2-3D.

К счастью, мы, инжинеры, знаем о методах уменьшения размерности.


#### Задание:
Получите 2 списока слов: 50 ближайших к слову офицер и 50 ближайших к слову женщина и соедините в 1.

In [0]:
words1 = <your code here>
words2 = <your code here>

In [0]:
words = words1 + words2

Получите вектора эмбеддингов

In [0]:
words1_vectors = <your code here>
words2_vectors = <your code here>

words_vectors = np.vstack((words2_vectors, words1_vectors))

In [0]:
# Test
assert isinstance(words_vectors, np.ndarray)
assert words_vectors.shape == (100, 32)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA
Простейшим методом уменьшения размерности является Principial Component Analysis (PCA).

В геометрических терминах PCA пытается найти оси, вдоль которых возникает большая часть дисперсии. «Естественные» оси, если хотите.

![alt text](https://camo.githubusercontent.com/a68740188733509d867957a6b37712fa6253d2fd/68747470733a2f2f6769746875622e636f6d2f79616e646578646174617363686f6f6c2f50726163746963616c5f524c2f7261772f6d61737465722f7965745f616e6f746865725f7765656b2f5f7265736f757263652f7063615f666973682e706e67)

Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing mean squared error:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$


$X \in \mathbb{R}^{n \times m}$ - object matrix (centered);

$W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;

$\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;

$n$ samples, $m$ original dimensions and $d$ target dimensions;

#### Задание:
Переведите вектора в 2d пространство с помощью PCA. Используйте "good old" sklearn api (fit, transform)
После чего нормальчуйте вектора так, чтобы они имели mean=0 и variance=1

In [0]:
from sklearn.decomposition import PCA



word1_vectors_pca = <your code here>

# Normalize
word_vectors_pca = <your code here>

In [0]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [0]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%pylab inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def draw_vectors(x_coords, y_coords, words=words):
    # display scatter plot
    plt.figure(figsize=(20, 15))
    plt.scatter(x_coords, y_coords, color=cm.rainbow([0.5] * 50 + [0.8] * 50))

    for label, x, y in zip(words, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    # plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    # plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.grid(True)
    plt.show()

In [0]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], words=words)

### 5. Визуализируем с помощью t-SNE

In [0]:
from sklearn.manifold import TSNE


word_tsne = <your code here>

# Normalize:
word_tsne = <your code here>

In [0]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], words=words)

### 6. Использование предобученной модели

Это заняло время, а? Теперь представьте, что вы можете обучение эмбедднингов слов life-sized (100~300D) на гигабайты текста: статьи в Википедии или посты в Твиттере.
К счастью, в настоящее время вы можете получить предварительно обученную модель эмбеддингов слов в 2 строки кода (без регистрации и смс).

In [0]:
# For en:
# import gensim.downloader as api
# model = api.load('glove-twitter-100')

# For russian

from gensim.models import *

# ru.vec was downloaded in the top of notebook
model300 = KeyedVectors.load_word2vec_format('ru.vec', binary=False)

In [0]:
model300.most_similar(positive=["человек", "офицер"], negative=["генерал"])

#### Задание:
Повторите эксперимент и постройте TSNE график

In [0]:
words1 = <your code here>
words2 = <your code here>
words = words1 + words2

words1_vectors = <your code here>

words2_vectors = <your code here>

words_vectors = np.vstack((words2_vectors, words1_vectors))

In [0]:
from sklearn.manifold import TSNE


word_tsne = <your code here>
word_tsne = <your code here>

In [0]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], words=words)

### 7. Анализ предложений
#### Задание:
1. lowercase phrase
2. tokenize phrase
3. average word vectors for all words in tokenized phrase


* skip words that are not in model's vocabulary
* if all words are missing from vocabulary, return zeros

In [0]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    vector = np.zeros([model300.vector_size], dtype='float32')
    
    <your code here>
    
    return vector

In [0]:
vector = get_phrase_embedding(" ".join(data_tok[0]))

assert np.allclose(vector[:15],
                   np.array([-0.09521323,  0.12494637,  0.07243977,  0.19425696,  0.09630267,
                              0.02568892,  0.04206766, -0.22367822, -0.12968796,  0.10239005,
                              0.01887055, -0.06602988, -0.07015807, -0.06562313, -0.11581734],
                              dtype=np.float32))

#### Задание:
Визуализируйте 20 случайных фраз

In [0]:
import numpy as np

In [0]:
# random choice
chosen_phrases = <your code here>

# compute vectors for chosen phrases
phrase_vectors = <your code here>

In [0]:
# Test
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model300.vector_size)

In [0]:
from sklearn.manifold import TSNE


word_tsne = <your code here>

# Normalize
word_tsne = <your code here>

draw_vectors(word_tsne[:, 0], word_tsne[:, 1], words=chosen_phrases)

### 8. Расстояние между высказываниями
В конце давайте напишем алгоритм, который позволит считать расстояния между высказываниями

In [349]:
data_vectors = np.array([get_phrase_embedding(" ".join(l)) for l in data_tok])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [0]:
import numpy as np


def cosine(matrix, vector):
    dotted = matrix.dot(vector)
    matrix_norms = np.linalg.norm(matrix, axis=1)
    vector_norm = np.linalg.norm(vector)
    matrix_vector_norms = np.multiply(matrix_norms, vector_norm)
    neighbors = np.divide(dotted, matrix_vector_norms)
    return neighbors


def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    vector = get_phrase_embedding(query)
    neighbors = <your code here>
    ids = <your code here>
    
    return <your code here>

In [0]:
print(" ".join(find_nearest(" ".join(data_tok[0]), 1)[0]))

### 9. Word Mover's Distance
#### по статье Matthew J. Kusner'а "From Word Embeddings to Document Distances" [1]

<img src="https://camo.githubusercontent.com/a15d483ec194469a9bea650fed26bf891b07ae94/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6d6b75736e65722f776d642f6d61737465722f666967312e706e67">

### Формулировка задачи определения сходства между двумя предложениями как задачи транспортной задачи:

Пусть $X \in \mathbb{R}^{d \times n}$ – матрица эмбеддингов,

$d$ – размерность эмбеддинга, $n$ - количество слов;

Вектор-документ в векторной модели: $d \in \mathbb{R}^n$ состоит из $c_i = \texttt{count}(word_i, doc)$

Нормированный вектор-документ: $d_i = \frac{c_i}{\sum_i c_i}$

Расстояние между словами: $\texttt{cost}(word_i, word_j) = ||x_i - x_j||_2$

Дано два документа, $d, d'$. Пусть  $T \in \mathbb{R}^{n \times n}$, $T_{ij} \ge 0$ – матрица потока показывает расстояния от каждого слова $d$ до $d'$.

Транспортная задача:

$\min_{T \ge 0} \sum_{i,j}^n T_{ij}\texttt{cost}(word_i, word_j) $

при условии:

$\sum_{j} T_{ij} = d_i$

$\sum_{i} T_{ij} = d'_j$.

Задача решается средствами линейного программирования.

In [0]:
s1 = 'Ученые обнаружили ископаемую ящерицу с парой теменных глаз'
s2 = 'У палеогеновой ящерицы нашли вторую пару глаз'
s3 = 'Apple через два года откажется от процессоров Intel'

distance = model300.wmdistance(s1, s2)
print ('distance between s1 and s2 = %.4f' % distance)

distance = model300.wmdistance(s1, s3)
print ('distance between s1 and s3 = %.4f' % distance)

distance = model300.wmdistance(s2, s3)
print ('distance between s2 and s3 = %.4f' % distance)

In [0]:
model300.init_sims(replace=True)

In [0]:
distance = model300.wmdistance(s1, s2)
print ('distance between s1 and s2 = %.4f' % distance)

distance = model300.wmdistance(s1, s3)
print ('distance between s1 and s3 = %.4f' % distance)

distance = model300.wmdistance(s2, s3)
print ('distance between s2 and s3 = %.4f' % distance)